<a href="https://colab.research.google.com/github/jlc2295/DSIRiverProject/blob/master/1_5_Improving_Edge_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import numpy as np
from IPython.display import Image
import matplotlib.pyplot as plt

# Trigger the authentication flow.
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=JDgWuaX5AOAyceHifg_z-beD1ds0PPW0wKhpk7Fhw30&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/3AGr9lXwFqeD_VXA4L94cLQqv8LsdIYTpQUyNd9HAoJWFkdbWwmMjXQ

Successfully saved authorization token.


In [ ]:
ee.Initialize()

In [ ]:
def get_images(path_list , row_list , satelite , start_date , end_date , max_cloud_percentage , months):
    
    # get image collection object
    coll = ee.ImageCollection(satelite)\
        .filterDate(start_date, end_date)\
        .filter(ee.Filter.inList('WRS_PATH', path_list))\
        .filter(ee.Filter.inList('WRS_ROW', row_list))\
        .filter(ee.Filter.lt('CLOUD_COVER' , max_cloud_percentage))\
        .filter(ee.Filter.calendarRange(months[0],months[1],'month'))    # just may data
    # get image_id's
    image_ids = list( map( lambda x : x['id'] , coll.getInfo()['features'] ) ) 
    
    # get image objects
    images = list( map( lambda x: ee.Image(x) , image_ids ) )
    
    return images

In [ ]:
#Lake Chapala Region
p = [29]
r = [46]
sat = 'LANDSAT/LC08/C01/T1_TOA'
sd = '2013-05-01'
ed = '2020-05-01'
cc= 10
months = [5 , 5] #Connor did months because greatest changes were seen during that time for his lake
image_list = get_images(p, r, sat, sd, ed, cc, months)

In [ ]:
#Hard code a bounding box
lake_bounding_box = ([-103.48,20.03],\
 [-102.63,20.03],\
 [-102.63,20.41],\
 [-103.48,20.41])

lake_region = ee.Geometry.Rectangle([-103.48 , 20.03 , -102.63 , 20.41])

In [ ]:
# this function clip to the lake
def clip(image):
    lake_region = ee.Geometry.Rectangle([-103.48 , 20.03 , -102.63 , 20.41])
    return image.clip(lake_region)

In [ ]:
# map the function to our image_list
image_list = list(map(clip, image_list))

Mask the Water

In [ ]:
# add ndwi and ndvi bands. For some reason, my NDWI is not working properly. (are they correct bands?)
#I think Arvind said the correct bands for NDWI. Double check/google.
def addNDWI(image):
    return image.addBands(image.normalizedDifference(['B5', 'B7']))

def addNDVI(image):
    return image.addBands(image.normalizedDifference(['B3', 'B5']))

In [ ]:
#add ndvi
image_list = list(map(addNDVI, image_list))        # set as 'nd'

In [ ]:
parameters = {'min': -1.0, # value which is mapped to 0
              'max': 1.0,  # value which is mapped to 255
              'dimensions': 768,   # size of the image 
              'bands': ['nd'],     # The bands we select
              'palette': ['green', 'white', 'blue'],  # only use a pallete when visualizing one.
              #'region': lake_region
}

lake_image = image_list[0]
display(Image(url = lake_image.getThumbUrl(parameters)))

Obtain water pixels only



In [ ]:
waterthreshold = .1;
image_ndvi = lake_image.select(['nd'])

watermask = image_ndvi.gte(waterthreshold)
fin = image_ndvi.updateMask(watermask)

In [ ]:
proj = fin.projection()
coord_space = fin.pixelCoordinates(proj)
#Creates a two band image containing the x and y coordinates of each pixel in the given projection.
latlon = fin.pixelLonLat()

latlon = latlon.updateMask(watermask)


coords = latlon.select(['longitude', 'latitude'])\
             .reduceRegion(\
 reducer = ee.Reducer.toList(),\
 geometry = lake_region,\
 scale = 30)


lat = ee.List(coords.get('latitude'))
lon = ee.List(coords.get('longitude'))

# zip them. Example: zip([1, 3],[2, 4]) --> [[1, 2], [3,4]]
point_list = lon.zip(lat)
mp = ee.Geometry.MultiPoint(point_list)

In [ ]:
latlon.bandNames().getInfo()

['longitude', 'latitude']

In [ ]:
mp.getInfo()

{'coordinates': [[-102.64954428528941, 20.32487737661044],
  [-102.64927479070417, 20.32487737661044],
  [-102.64900529611893, 20.32487737661044],
  [-102.64954428528941, 20.325146871195678],
  [-102.64927479070417, 20.325146871195678],
  [-102.64954428528941, 20.325416365780914],
  [-102.64927479070417, 20.325416365780914],
  [-102.64145944773233, 20.36718802649247],
  [-102.64900529611893, 20.03031979494765],
  [-102.64954428528941, 20.030589289532887],
  [-102.64927479070417, 20.030589289532887],
  [-102.64900529611893, 20.030589289532887],
  [-102.64981377987463, 20.030858784118124],
  [-102.64954428528941, 20.030858784118124],
  [-102.64927479070417, 20.030858784118124],
  [-102.64900529611893, 20.030858784118124],
  [-102.6487358015337, 20.030858784118124],
  [-102.64846630694846, 20.030858784118124],
  [-102.64819681236322, 20.030858784118124],
  [-102.64900529611893, 20.031128278703356],
  [-102.6487358015337, 20.031128278703356],
  [-102.64846630694846, 20.031128278703356],
  

Get convex hull

In [ ]:
convex_hull = mp.convexHull()

In [ ]:
type(convex_hull)

ee.geometry.Geometry

In [ ]:
# This Returns the perimeter of the convex hull in meters
convex_hull.perimeter().getInfo()

249833.25609216743

In [ ]:
# I would Like to make my own class for this.
import folium 
class Map_Display():

    def __init__(self , location , zoom , height  ):
      self.m =  folium.Map(location= location , zoom_start = zoom)


    def add_ee_layer(self, ee_image_object, vis_params, name):
        map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
        folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = "Map Data © Google Earth Engine",
        name = name,
        overlay = True,
        control = True
        ).add_to(self.m)

    def add_child(self , child_object):
      self.m.add_child(child_object)

    def show_map(self):
      display(self.m)

In [ ]:
# Test it out
md = Map_Display([20.08, -103.16] , zoom = 10 , height = 500)
#md = Map_Display([4149 , 2524] , zoom = 12 , height = 500)


# Set visualization parameters.
vis_params_W = {
  'min': 0.5,
  'max': 1.0,
  #'palette': ['00FFFF', '0000FF']
}

vis_params_V = {
  'min': -1,
  'max': 1,
  #'palette': ['#d73027', '#f46d43', '#fdae61','#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
  #'palette': ['blue', 'white', 'green']
    #'palette': ['blue' , 'white' , 'green']
  }

In [ ]:
md.show_map()

In [ ]:
folium.GeoJson(
            data = convex_hull.getInfo(),
            name = 'convex_hull',
            overlay = True,
            control = True,
        ).add_to(md.m)

In [ ]:
md.show_map()

Edge Detection

In [ ]:
zc = ee.Algorithms.CannyEdgeDetector(
  image = watermask,
  threshold = 0.5)

In [ ]:
parameters = {'min': 0,
              'max': 1,
              'dimensions': 768,
              'palette': ['white', 'grey', 'blue'],
}
Image(url = zc.getThumbUrl(parameters))

In [ ]:

new_proj = zc.projection()
new_coord_space = zc.pixelCoordinates(new_proj)
#Creates a two band image containing the x and y coordinates of each pixel in the given projection.
new_latlon = zc.pixelLonLat()

new_latlon = new_latlon.updateMask(watermask)


new_coords = new_latlon.select(['longitude', 'latitude'])\
             .reduceRegion(\
 reducer = ee.Reducer.toList(),\
 geometry = lake_region,\
 scale = 30)


new_lat = ee.List(new_coords.get('latitude'))
new_lon = ee.List(new_coords.get('longitude'))

# zip them. Example: zip([1, 3],[2, 4]) --> [[1, 2], [3,4]]
new_point_list = new_lon.zip(new_lat)
new_mp = ee.Geometry.MultiPoint(new_point_list)

In [ ]:
new_mp.getInfo()

{'coordinates': [[-102.64954428528941, 20.32487737661044],
  [-102.64927479070417, 20.32487737661044],
  [-102.64900529611893, 20.32487737661044],
  [-102.64954428528941, 20.325146871195678],
  [-102.64927479070417, 20.325146871195678],
  [-102.64954428528941, 20.325416365780914],
  [-102.64927479070417, 20.325416365780914],
  [-102.64145944773233, 20.36718802649247],
  [-102.64900529611893, 20.03031979494765],
  [-102.64954428528941, 20.030589289532887],
  [-102.64927479070417, 20.030589289532887],
  [-102.64900529611893, 20.030589289532887],
  [-102.64981377987463, 20.030858784118124],
  [-102.64954428528941, 20.030858784118124],
  [-102.64927479070417, 20.030858784118124],
  [-102.64900529611893, 20.030858784118124],
  [-102.6487358015337, 20.030858784118124],
  [-102.64846630694846, 20.030858784118124],
  [-102.64819681236322, 20.030858784118124],
  [-102.64900529611893, 20.031128278703356],
  [-102.6487358015337, 20.031128278703356],
  [-102.64846630694846, 20.031128278703356],
  

In [ ]:
geometries = mp.geometries()
pt1 = geometries.get(0).getInfo()

print(geometries.length().getInfo())
#
# Adding the point breks it
#geometries = geometries.add(new_pt)

print(geometries.length().getInfo())
lr = ee.Geometry.LinearRing(geometries)
pg = ee.Geometry.Polygon(geometries)

1174411
1174411


In [ ]:
pg.perimeter().getInfo()

KeyboardInterrupt: ignored

In [ ]:
# Test it out
md2 = Map_Display([20.08, -103.16] , zoom = 12 , height = 500)
#md = Map_Display([4149 , 2524] , zoom = 12 , height = 500)


# Set visualization parameters.
vis_params_W = {
  'min': 0.5,
  'max': 1.0,
  #'palette': ['00FFFF', '0000FF']
}

vis_params_V = {
  'min': -1,
  'max': 1,
  #'palette': ['#d73027', '#f46d43', '#fdae61','#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']
  #'palette': ['blue', 'white', 'green']
    #'palette': ['blue' , 'white' , 'green']
  }

In [ ]:
folium.GeoJson(
            data = pg.getInfo(),
            name = 'pg',
            overlay = True,
            control = True,
        ).add_to(md2.m)

KeyboardInterrupt: ignored

In [ ]:
md2.show_map()

Other methods for Edge Detection


*   Directional: once a pixel part of the edge is detected on a particular side of the previous pixel, only check pixels on that side until no pixels detected, then check all sides of pixel
*   zero crossing function: takes any pixel, divides image to water and non water (GEE api): divide image into water and nw regions, see surrounding pixels, common edge, surrounded with water on all 4 sides? only at edge where pixel is surrounded by both land and water on both sides, that would be considered a 1 (1 is edge, build your edge that way)

